### Tool to check weather in a city

In [8]:
import json
import os
from dotenv import load_dotenv
from openai import OpenAI

# `weather.py` lives next to this notebook in `week2/`
from weather import get_weather

load_dotenv(override=True)

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
OPENROUTER_BASE_URL = os.getenv("OPENROUTER_BASE_URL", "https://openrouter.ai/api/v1")
MODEL = os.getenv("OPENROUTER_MODEL", "openai/gpt-4o-mini")

client = OpenAI(api_key=OPENROUTER_API_KEY, base_url=OPENROUTER_BASE_URL)

In [9]:
weather_tool_schema = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current weather conditions for a city.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "City name, e.g. Nairobi, Kenya",
                }
            },
            "required": ["city"],
        },
    },
}


def call_tool(tool_name: str, arguments: dict):
    if tool_name == "get_weather":
        return get_weather(**arguments)
    raise ValueError(f"Unknown tool: {tool_name}")

In [10]:
from IPython.display import Markdown


SYSTEM_PROMPT = """
You are a helpful weather assistant.
If you need live weather for a city, call the get_weather tool.
After you get tool results, write a short report with:
- location
- observed time
- temperature and feels-like
- humidity, wind, precipitation
- a plain-English weather description
Keep it concise.
""".strip()


def weather_agent(question: str) -> str:
    if not OPENROUTER_API_KEY:
        raise ValueError("Missing OPENROUTER_API_KEY in your environment (.env)")

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": question},
    ]

    resp = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=[weather_tool_schema],
        tool_choice="auto",
    )

    msg = resp.choices[0].message
    tool_calls = getattr(msg, "tool_calls", None)

    # If the model didn't call tools, just return its response.
    if not tool_calls:
        return msg.content or ""

    # Add the assistant message WITH tool_calls in plain dict form.
    assistant_tool_calls = []
    for tc in tool_calls:
        assistant_tool_calls.append(
            {
                "id": tc.id,
                "type": "function",
                "function": {
                    "name": tc.function.name,
                    "arguments": tc.function.arguments,
                },
            }
        )

    messages.append(
        {
            "role": "assistant",
            "content": msg.content,
            "tool_calls": assistant_tool_calls,
        }
    )

    # Execute tool calls and provide results back.
    for tc in tool_calls:
        args = json.loads(tc.function.arguments or "{}")
        tool_result = call_tool(tc.function.name, args)
        messages.append(
            {
                "role": "tool",
                "tool_call_id": tc.id,
                "name": tc.function.name,
                "content": json.dumps(tool_result, ensure_ascii=False),
            }
        )

    final = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=[weather_tool_schema],
    )
    return final.choices[0].message.content or ""

In [6]:
weather_agent("What's the weather in Ruiru right now?")

'**Weather Report for Ruiru, Kenya:**\n\n- **Observed Time:** February 22, 2026, at 18:15\n- **Temperature:** 20.0°C (Feels like 21.2°C)\n- **Humidity:** 80%\n- **Wind:** 7.1 kph\n- **Precipitation:** 0.7 mm\n\n**Weather Description:** Thunderstorms are currently affecting Ruiru, bringing a warm and humid atmosphere.'

In [11]:
import gradio as gr


def weather_agent_stream(question: str):
    """Streaming version of weather_agent() for Gradio."""
    if not OPENROUTER_API_KEY:
        raise ValueError("Missing OPENROUTER_API_KEY in your environment (.env)")

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": question},
    ]

    # 1) Decide whether to call tools (non-stream)
    resp = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=[weather_tool_schema],
        tool_choice="auto",
    )

    msg = resp.choices[0].message
    tool_calls = getattr(msg, "tool_calls", None)

    # If no tool calls, stream the assistant's direct response.
    if not tool_calls:
        direct = msg.content or ""
        yield direct
        return

    # Add assistant tool call message (dict) then tool results.
    assistant_tool_calls = [
        {
            "id": tc.id,
            "type": "function",
            "function": {"name": tc.function.name, "arguments": tc.function.arguments},
        }
        for tc in tool_calls
    ]

    messages.append({"role": "assistant", "content": msg.content, "tool_calls": assistant_tool_calls})

    for tc in tool_calls:
        args = json.loads(tc.function.arguments or "{}")
        tool_result = call_tool(tc.function.name, args)
        messages.append(
            {
                "role": "tool",
                "tool_call_id": tc.id,
                "name": tc.function.name,
                "content": json.dumps(tool_result, ensure_ascii=False),
            }
        )

    # 2) Stream the final report
    stream = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True,
        tools=[weather_tool_schema],
    )

    out = ""
    for chunk in stream:
        out += chunk.choices[0].delta.content or ""
        yield out


gr.Interface(
    fn=weather_agent_stream,
    inputs=gr.Textbox(label="City or question", placeholder="What's the weather in Nairobi?"),
    outputs=gr.Markdown(label="Weather report"),
    title="Weather Agent (streaming)",
    examples=[
        "What's the weather in Nairobi right now?",
        "Current weather in New York",
        "Weather in Tokyo today",
    ],
    flagging_mode="never",
).launch()

c:\Users\maina\miniconda3\envs\llms\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7887
* To create a public link, set `share=True` in `launch()`.
